In [273]:
import numpy as np # фреймы данных
import pandas as pd # графики
import matplotlib as mpl
# стили и шаблоны графиков на основе matplotlib
import seaborn as sns
# для таймера
import time
# загрузка пакетов: модели ------------------------------------------------- # SVM
from sklearn.svm import SVC
# логистическая рагрессия
from sklearn.linear_model import LogisticRegression # стандартизация
from sklearn.preprocessing import StandardScaler
# метод главных компонент
from sklearn.decomposition import PCA
# конвейеры
from sklearn.pipeline import make_pipeline
# перекрёстная проверка и метод проверочной выборки
from sklearn.model_selection import cross_val_score, train_test_split # для перекрёстной проверки и сеточного поиска
from sklearn.model_selection import KFold, GridSearchCV
# сводка по точности классификации
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier


In [274]:
# константы
# ядро для генератора случайных чисел
my_seed = 11
# создаём псевдоним для короткого обращения к графикам
plt = mpl.pyplot
# настройка стиля и отображения графиков
# примеры стилей и шаблонов графиков:
# http://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html mpl.style.use('seaborn-whitegrid')
sns.set_palette("Set2")
# раскомментируйте следующую строку, чтобы посмотреть палитру
# sns.color_palette("Set2")

In [275]:
DF_raw = pd.read_csv('https://raw.githubusercontent.com/ania607/ML/main/data/winequality-white_for_lab.csv')

DF_raw = DF_raw[['fixed_acidity','volatile_acidity','citric_acid','residual_sugar','chlorides','free_sulfur_dioxide',
                'total_sulfur_dioxide','density','pH','sulphates','alcohol','Y']]

In [276]:
# наблюдения для моделирования
DF = DF_raw.sample(frac = 0.85, random_state = my_seed) # отложенные наблюдения
DF_predict = DF_raw.drop(DF.index)

In [277]:
DF_raw.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,Y
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,1
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,1
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,1


Все данные являются количественными

**<h3>Стандартизация и переход к главным компонентам</h3>**

In [278]:
# стандартизация
sc = StandardScaler()
X_train_std = sc.fit_transform(DF.iloc[:, :-1].values)
# оцениваем объяснённую главными компонентами дисперсию
pca = PCA()
X_train_pca = pca.fit_transform(X_train_std)
# считаем доли объяснённой дисперсии
frac_var_expl = pca.explained_variance_ratio_
print('Доли объяснённой дисперсии по компонентам в PLS:\n',
np.around(frac_var_expl, 3),
'\nОбщая сумма долей:', np.around(sum(frac_var_expl), 3))

Доли объяснённой дисперсии по компонентам в PLS:
 [0.291 0.144 0.112 0.092 0.089 0.086 0.066 0.055 0.038 0.027 0.002] 
Общая сумма долей: 1.0


Таким образом, первые две главные компоненты объясняют 40% разброса 11 объясняющих переменных

In [279]:
X_train = DF[['fixed_acidity','volatile_acidity','citric_acid','residual_sugar','chlorides','free_sulfur_dioxide',
                'total_sulfur_dioxide','density','pH','sulphates','alcohol']]
y_train = DF.Y

**<h3>Модель 1</h3>**

In [263]:
pipe_svc = make_pipeline(StandardScaler(), SVC(random_state=my_seed))
# настроим параметры SVM с помощью сеточного поиска

param_grid = [{'svc__C': [0.1, 100, 80],
              'svc__gamma': [1, 0.1, 0.2, 0.001],
              'svc__kernel': ['rbf']}]

# разбиения для перекрёстной проверки
kfold = KFold(n_splits=5, random_state=my_seed, shuffle=True)
gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, scoring='accuracy', refit=True, cv=kfold,
n_jobs=-1)
# таймер
tic = time.perf_counter()
# запускаем сеточный поиск
gs = gs.fit(X_train, y_train)
# таймер
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

Сеточный поиск занял 50.93 секунд


In [264]:
# точность лучшей модели
np.around(gs.best_score_, 3)


0.801

In [265]:
gs.best_estimator_.get_params()

{'memory': None,
 'steps': [('standardscaler', StandardScaler()),
  ('svc', SVC(C=100, gamma=1, random_state=11))],
 'verbose': False,
 'standardscaler': StandardScaler(),
 'svc': SVC(C=100, gamma=1, random_state=11),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'svc__C': 100,
 'svc__break_ties': False,
 'svc__cache_size': 200,
 'svc__class_weight': None,
 'svc__coef0': 0.0,
 'svc__decision_function_shape': 'ovr',
 'svc__degree': 3,
 'svc__gamma': 1,
 'svc__kernel': 'rbf',
 'svc__max_iter': -1,
 'svc__probability': False,
 'svc__random_state': 11,
 'svc__shrinking': True,
 'svc__tol': 0.001,
 'svc__verbose': False}

**<h3>Модель 2</h3>**

In [266]:
pipe_tree = make_pipeline(
      StandardScaler(),
      RandomForestClassifier(random_state=my_seed),
)


# настроим параметры SVM с помощью сеточного поиска

param_grid = {
    'randomforestclassifier__n_estimators': [60],
    'randomforestclassifier__max_depth': [20, 30],
    'randomforestclassifier__max_features': [20, 25],
    'randomforestclassifier__criterion': ['entropy'],
    'randomforestclassifier__min_samples_split': [3],
}

# разбиения для перекрёстной проверки
kfold = KFold(n_splits=10, random_state=my_seed, shuffle=True)
gs2 = GridSearchCV(estimator=pipe_tree, param_grid=param_grid, scoring='accuracy', refit=True, cv=kfold,
n_jobs=-1)
# таймер
tic = time.perf_counter()
# запускаем сеточный поиск
gs2 = gs2.fit(X_train, y_train)
# таймер
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

Сеточный поиск занял 47.57 секунд


In [267]:
# точность лучшей модели
np.around(gs2.best_score_, 3)

0.835

In [268]:
 gs2.best_estimator_.get_params()

{'memory': None,
 'steps': [('standardscaler', StandardScaler()),
  ('randomforestclassifier',
   RandomForestClassifier(criterion='entropy', max_depth=30, max_features=20,
                          min_samples_split=3, n_estimators=60, random_state=11))],
 'verbose': False,
 'standardscaler': StandardScaler(),
 'randomforestclassifier': RandomForestClassifier(criterion='entropy', max_depth=30, max_features=20,
                        min_samples_split=3, n_estimators=60, random_state=11),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'randomforestclassifier__bootstrap': True,
 'randomforestclassifier__ccp_alpha': 0.0,
 'randomforestclassifier__class_weight': None,
 'randomforestclassifier__criterion': 'entropy',
 'randomforestclassifier__max_depth': 30,
 'randomforestclassifier__max_features': 20,
 'randomforestclassifier__max_leaf_nodes': None,
 'randomforestclassifier__max_samples': None,
 'randomforestclassifier__min_impurity

Максимальную метрику Acc = 83,5% на обучающей выборке получилось выбить с помощью второй модели. Получить Acc больше этого значения, к сожалению, не получилось

In [269]:
score = list()
score_models = list()
score.append(np.around(gs.best_score_, 3))
score_models.append('model_1')

score.append(np.around(gs2.best_score_, 3))
score_models.append('model_2')

**<h3>Прогноз на отложенные наблюдения по лучшей модели</h3>**

In [270]:
pd.DataFrame({'Модель' : score_models, 'Acc' : score})

,Модель,Acc
0,model_1,0.801
1,model_2,0.835


In [272]:
# прогноз с помощью лучшей модели (модель 2)
y_hat = gs2.best_estimator_.predict(X=DF_predict[['fixed_acidity','volatile_acidity','citric_acid','residual_sugar','chlorides','free_sulfur_dioxide',
                'total_sulfur_dioxide','density','pH','sulphates','alcohol']])
# точность
# характеристики точности
print(classification_report(DF_predict.Y, y_hat))

              precision    recall  f1-score   support

           0       0.80      0.69      0.74       237
           1       0.86      0.92      0.89       498

    accuracy                           0.85       735
   macro avg       0.83      0.81      0.82       735
weighted avg       0.84      0.85      0.84       735



На тестовой выборке метрика Acc оказалась равна 85%